<div class="alert alert-block alert-warning"><strong>If you have this notebook as a local copy on your platform, it may become outdated. <a href="https://dataplatform.cloud.ibm.com/exchange/public/entry/view/029d77a73d72a4134c81383d6f020f6f?context=cpdaas&audience=wdp">Download the latest version of this notebook</a> or download the <a href="https://dataplatform.cloud.ibm.com/exchange/public/entry/view/cab78523832431e767c41527a42a6727?context=cpdaas?context=cpdaas&audience=wdp">latest version of the project</a>.</strong></div>

# Part 2 - WML Federated Learning with MNIST for Party 

### Learning Goals

When you complete the Part 2 - WML Federated Learning with MNIST for Party, you should know how to:

- Load the data that you intend to use in the Federated Learning experiment.
- Install IBM Federated Learning libraries.
- Define a data handler. For more details on data handlers, see <a href = "https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fl-cus-dh.html?audience=wdp&context=cpdaas" target="_blank" rel="noopener no referrer">Customizing the data handler</a>.
- Configure the party to train data with the aggregator.

<div class="alert alert-block alert-info">This notebook is intended to be run by the administrator or connecting party of the Federated Learning experiment.
</div>

## Table of Contents

1. [Load the data](#load)<br>
2. [Install Federated Learning libraries](#install)<br>
3. [Define a Data Handler](#data-handler)<br>
4. [Configure the party](#config)<br>
5. [Train with Federated Learning](#train)<br>
6. [Summary](#summary)

<div class="alert alert-block alert-warning">Before you run this notebook, you must have already run <a href = "https://dataplatform.cloud.ibm.com/exchange/public/entry/view/029d77a73d72a4134c81383d6f020f6f?context=cpdaas">Part 1 - WML Federated Learning with MNIST for Admin</a>). If you have not, open the notebook and run through that notebook first.
</div>

<a id = "load"></a>
## 1. Load the data

### Paste Variables From Admin Notebook

Paste in the ID credentials you got from the end of the Part 1 notebook. If you have not run through Part 1, open the notebook and run through it first.

In [ ]:
WML_SERVICES_HOST = ''
IAM_APIKEY = ''
RTS_ID = ''
TRAINING_ID = ''

<a id = "1.1"></a>
### 1.1 Download MNIST handwritten digits dataset

As the party, you must provide the dataset that you will use to train the Federated Learning model. In this tutorial, a dataset is provided by default, the MNIST handwritten digits dataset.

In [ ]:
import requests

dataset_resp = requests.get("https://api.dataplatform.cloud.ibm.com/v2/gallery-assets/entries/903188bb984a30f38bb889102a1baae5/data",
                            allow_redirects=True)

f = open('MNIST-pkl.zip', 'wb')
f.write(dataset_resp.content)
f.close()

In [ ]:
import zipfile
import os

with zipfile.ZipFile("MNIST-pkl.zip","r") as file:
    file.extractall()
    
!ls -lh

<a id = "install"></a>
## 2. Install Federated Learning libraries

In this section, we will install the necessary libraries and other packages to call for Federated Learning with the Python client.

<a id = "2.1"></a>
### 2.1 Install the IBM WML SDK with FL

This installs the IBM Watson Machine Learning CLI along with the whole software development package with Federated Learning.

In [ ]:
# import sys
# !{sys.executable} -m pip install --upgrade ibm-watson-machine-learning

<a id = "2.2"></a>
### 2.2 Install the libraries

In [ ]:
#!{sys.executable} -m pip install environs parse websockets jsonpickle pandas pytest pyYAML requests pathlib2 psutil setproctitle tabulate lz4 opencv-python gym ray==0.8.0 cloudpickle==1.3.0 image

<a id = "2.3"></a>
### 2.3 Install the frameworks

In [ ]:
#!{sys.executable} -m pip install tensorflow==2.4.1 scikit-learn==0.23.2 keras==2.2.4 numpy==1.19.2 scipy==1.4.1

In [ ]:
!pip install --upgrade ibm-watson-machine-learning

!pip install tensorflow==2.4.3 \
torch==1.7.1 \
scikit-learn==0.23.2 \
scipy==1.6.3 \
PyYAML \
aif360==0.3.0 \
ddsketch \
diffprivlib \
environs \
h5py \
image \
jsonpickle==1.4.1 \
keras==2.2.4 \
numcompress \
numpy \
pandas \
parse \
pathlib2 \
pillow \
pytest \
requests \
skorch \
torchvision==0.2.2 \
websockets \
psutil \
setproctitle \
lz4 \
gym \
ray

<a id = "2.4"></a>
### 2.4 Import the Party

The following code imports the package for the party, and ensures that it is loaded.

In [ ]:
import ibmfl.party_env_validator
from ibmfl.party.party import Party

<a id = "data-handler"></a>
## 3. Define a Data Handler

The party should run a data handler to ensure that their datasets are in compatible format and consistent. In this tutorial, an example data handler for the MNIST dataset is provided. 

For more details on data handlers, see <a href = "https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fl-cus-dh.html?audience=wdp&context=cpdaas" target="_blank" rel="noopener no referrer">Customizing the data handler</a>.

This data handler is written to the local working directory of this notebook

In [ ]:
import requests
data_handler_content_resp = requests.get("https://github.com/IBMDataScience/sample-notebooks/raw/master/Files/mnist_keras_data_handler.py",
                                  headers={"Content-Type": "application/octet-stream"},
                                      allow_redirects=True)

f = open('mnist_keras_data_handler.py', 'wb')
f.write(data_handler_content_resp.content)
f.close()

### Verify Data Handler Exists

In [ ]:
!ls -lh

<a id = "config"></a>
## 4. Configure the party

Each party must run their party configuration file to call out to the aggregator. Here is an example of a party configuration.

Because you had already defined the training ID, RTS ID and data handler in the previous sections of this notebook, and the local training and protocol handler are all defined by the SDK, you will only need to define the information for the dataset file under `["data"]["info"]`. 

In this tutorial, the data path is already defined as we have loaded the examplar MNIST dataset from previous sections.

In [ ]:
from pathlib import Path
working_dir = !pwd
pwd = working_dir[0]

party_config = {
  "aggregator": {
    "ip": WML_SERVICES_HOST + "/ml/v4/trainings/" + TRAINING_ID
  },
  "connection": {
    "info": {
      "id": RTS_ID,
    }
  },
  "data": {
    "info": {
      "train_file": pwd + "/mnist-keras-train.pkl",
      "test_file": pwd + "/mnist-keras-test.pkl"
    },
    "name": "MnistTFDataHandler",
    "path": pwd + "/mnist_keras_data_handler.py"
  },
  "local_training": {
    "name": "LocalTrainingHandler",
    "path": "ibmfl.party.training.local_training_handler"
  },
  "protocol_handler": {
    "name": "PartyProtocolHandler",
    "path": "ibmfl.party.party_protocol_handler"
  }
}

In [ ]:
print(party_config)

<a id = "train"></a>
## 5. Connect and train with Federated Learning

Here you can finally connect to the aggregator to begin training.

#### Obtain Cloud Authentication Token

In [ ]:
from ibm_watson_machine_learning import APIClient


wml_credentials = {
    "apikey": IAM_APIKEY,
    "url": "https://" + WML_SERVICES_HOST
}

wml_client = APIClient(wml_credentials)
IAMTOKEN = "Bearer " + wml_client.wml_token
print(IAMTOKEN)


### 5.1 Establish Connection To Aggregator

In [ ]:
p = Party( config_dict = party_config, token = IAMTOKEN )

After the message "Received Heartbeat from Aggregator" appears, then the Party is ready to start.

### 5.2 Start Training

In [ ]:
p.start()

<a id = "summary"></a>
## Summary

Congratulations! You have learned to:

1. Start a Federated Learning experiment
2. Load a template model
3. Create an RTS and launch the experiment job
4. Load a dataset for training
5. Define the data handler
6. Configure the party
7. Connect to the aggregator
8. Train your Federated Learning model

### Learn more

- For more details about setting up Federated Learning, terminology, and running Federated Learning from the UI, see <a href = "https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fed-lea.html?audience=wdp" target="_blank" rel="noopener no referrer">Federated Learning documentation</a> for Cloud.
- For more information on a Keras model template, see their documentation <a href = "https://www.tensorflow.org/tutorials/quickstart/advanced" target="_blank" rel="noopener no referrer">here</a>.

# <hr>
Copyright © 2020, 2021 IBM. This notebook and its source code are released under the terms of the MIT License.
<br>
 
<div style="background:#F5F7FA; height:110px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Love this notebook? </span>
<span style="font-size:15px;color:#152935;float:right;margin-right:40px;">Don't have an account yet?</span><br>
<span style="color:#5A6872;">Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style="border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;"><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
</div>